In [ ]:
import requests
from bs4 import BeautifulSoup
import math
import csv
import re
from tqdm import tqdm

# Target URL format with placeholder for pagination
target_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={}&location={}&start={}"


def text_clean(text):
    text = re.sub(r"\n\n+", "\n\n", text)
    text = re.sub(r"\t+", "\t", text)
    text = re.sub(r"\s+", " ", text)
    return text


# Define a user agent
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "max-age=0",
}

# List to store job IDs
job_ids = []
job_data = []

# Calculate number of pages to scrape (25 jobs per page)
total_jobs = 200  # Example number
pages = math.ceil(total_jobs / 25)

# Get all job IDs
for i in tqdm(range(0, pages)):
    res = requests.get(
        target_url.format("senior Data scientist", "remote", i * 25), headers=headers
    )
    soup = BeautifulSoup(res.text, "html.parser")
    jobs_on_page = soup.find_all("li")

    for job in jobs_on_page:
        try:
            job_id = (
                job.find("div", {"class": "base-card"})
                .get("data-entity-urn")
                .split(":")[-1]
            )
            job_ids.append(job_id)
        except:
            continue

# Get details for each job
job_details_url = "https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}"
for job_id in tqdm(job_ids):
    resp = requests.get(job_details_url.format(job_id), headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    job_info = {}
    try:
        job_info["company"] = (
            soup.find("div", {"class": "top-card-layout__card"})
            .find("a")
            .find("img")
            .get("alt")
        )
        job_info["title"] = (
            soup.find("div", {"class": "top-card-layout__entity-info"})
            .find("a")
            .text.strip()
        )
        # job_info["level"] = soup.find("ul", {"class": "description__job-criteria-list"}).find("li").text.replace("Seniority level", "").strip()

        # Get all li elements in the list
        criteria_list = soup.find(
            "ul", {"class": "description__job-criteria-list"}
        ).find_all("li")

        # Field names in order they typically appear
        field_names = ["level", "employment_type", "job_function", "industries"]
        field_labels = [
            "Seniority level",
            "Employment type",
            "Job function",
            "Industries",
        ]

        criteria_list = soup.find(
            "ul", {"class": "description__job-criteria-list"}
        ).find_all("li")

        # Process each field
        for i, field in enumerate(field_names):
            if i < len(criteria_list):
                job_info[field] = (
                    criteria_list[i].text.replace(field_labels[i], "").strip()
                )
            else:
                job_info[field] = ""
        job_info["description"] = text_clean(
            soup.find(
                "div", {"class": "description__text description__text--rich"}
            ).text.strip()
        )
        job_data.append(job_info)
    except:
        continue

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 80/80 [00:31<00:00,  2.53it/s]


In [159]:
import pandas as pd

df = pd.DataFrame.from_dict(job_data)
df.to_csv("linkedin_jobs.csv", index=False, encoding="utf-8-sig")

In [92]:
# import pprint
# pprint.pprint(df.iloc[0,-1])

In [8]:
import random
import time
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd
from tqdm import tqdm
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

from genu.Job_agent.config import LINKEDIN_JOB_SEARCH_URL, HEADERS
from genu.utils import text_clean

persist_directory = "../data/job_data/vectorstore"

# # Later, to load the saved vector store:
# loaded_vectorstore = Chroma(
#     persist_directory=f"{persist_directory}_chroma",
#     embedding_function=OpenAIEmbeddings(),
# )

ImportError: cannot import name 'LINKEDIN_JOB_SEARCH_URL' from 'genu.Job_agent.config' (/home/alireza/projects/GenU/genu/Job_agent/config.py)

In [ ]:
# # Get all documents
# all_docs = loaded_vectorstore.get()

# # Access the documents
# documents = all_docs["documents"]  # List of document contents
# metadatas = all_docs["metadatas"]  # List of document metadata
# ids = all_docs["ids"]  # List of document IDs

# # Print first few documents with their metadata
# for i in range(min(5, len(documents))):
#     print(f"Document {i+1} (ID: {ids[i]}):")
#     print(f"Content: {documents[i]}")
#     print(f"Metadata: {metadatas[i]}")
#     print("-" * 50)

# # Get total count
# print(f"Total documents: {len(documents)}")

In [ ]:
# # Get similar documents to a query
# query = "Optum"
# similar_docs = loaded_vectorstore.similarity_search(
#     query, k=5
# )  # k is number of results

# # Print the retrieved documents
# for i, doc in enumerate(similar_docs):
#     print(f"Result {i+1}:")
#     print(f"Content: {doc.page_content}")
#     print(f"Metadata: {doc.metadata}")
#     print("-" * 50)

In [11]:
from langchain.vectorstores import FAISS

persist_directory = "data/job_data/vectorstore"
loaded_vectorstore = FAISS.load_local(
    f"../{persist_directory}_faiss",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)
# For FAISS

print("FAISS vectorstore count:", len(loaded_vectorstore.index_to_docstore_id))

FAISS vectorstore count: 24


In [12]:
loaded_vectorstore.similarity_search("price optimization", k=2)

[Document(id='1f6e9753-7e05-41cd-9784-dde4b259831d', metadata={'company': 'Harnham', 'title': 'Director, Data Science', 'level': 'Director', 'employment_type': 'Full-time', 'job_function': 'Analyst', 'industries': 'Financial Services', 'link': 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4233084632', 'date': '2025-05-26'}, page_content='Director, Data Science \n Harnham \n Director of Data ScienceIndustry Leader – Financial Services$195,000 - $225,000 base salary + BonusRemote – EST/CST preferred The Company:A fast-growing, mission-driven organization transforming the financial services space through advanced analytics and AI-powered solutions. With a strong focus on ethical data use and measurable impact, they’re building out a modern data organization to deliver scalable, production-grade ML across marketing, sales, CX, and operations.The Role – Director of Data Science:This is an ideal opportunity for a highly technical leader ready to take the next step into a Director-

In [ ]:
from pprint import pprint

pprint(loaded_vectorstore.similarity_search("price optimization", k=5)[0].page_content)

('Senior Data Analyst- Merchandising \n'
 ' Quince \n'
 ' OUR STORY Quince was started to challenge the existing idea that nice '
 'things should cost a lot. Our mission was simple: create an item of equal or '
 'greater quality than the leading luxury brands and sell them at a much lower '
 'price.OUR VALUES Customer First. Customer satisfaction is our highest '
 'priority.High Quality. True quality is a combination of premium materials '
 'and high production standards that everyone can feel good about.Essential '
 "design. We don't chase trends, and we don't sell everything. We're expert "
 'curators that find the very best and bring it to you at the lowest '
 'prices.Always a better deal. Through innovation and real price transparency '
 'we want to offer the best deal to both our customers and our factory '
 "partners.Environmentally and Socially conscious. We're committed to "
 'sustainable materials and sustainable production methods. That means a '
 'cleaner environment and fai

In [150]:
import requests
from bs4 import BeautifulSoup
import math
import re
from tqdm import tqdm

# Target URL format with placeholder for pagination
# target_url = "https://www.linkedin.com/jobs/search/?currentJobId=4216706661&f_TPR=r604800&f_WT=2&geoId=103644278&keywords=senior%20data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true"
target_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={}&location={}&start={}"
# List to store job IDs
job_ids = []
job_data = []

# Calculate number of pages to scrape (25 jobs per page)
total_jobs = 500  # Example number
pages = math.ceil(total_jobs / 25)

# Get all job IDs
for i in tqdm(range(0, pages)):
    res = requests.get(target_url.format("Senior data scientist", "remote", i * 25))
    soup = BeautifulSoup(res.text, "html.parser")
    jobs_on_page = soup.find_all("li")

    for job in jobs_on_page:
        try:
            job_id = int(
                job.find("div", {"class": "base-card"})
                .get("data-entity-urn")
                .split(":")[-1]
            )
            # print(f"Found job ID: {job_id}")
            job_ids.append(job_id)
        except Exception as e:
            print(f"Error processing job: {e}")
            continue

print(len(set(job_ids)))
print(len(list(job_ids)))

100%|██████████| 20/20 [00:16<00:00,  1.18it/s]

200
200


In [147]:
target_url.format("Python", "New York", i * 25)

'https://www.linkedin.com/jobs/search/?currentJobId=4216706661&f_TPR=r604800&f_WT=2&geoId=103644278&keywords=senior%20data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true'

In [18]:
from genu.Job_agent.config import HEADERS, LINKEDIN_JOB_SEARCH_PARAMS


def linkedin_link_constructor(search_params: list[dict[str, str]]):

    target_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={}&location={}&f_TPR={}"

    url_list = []
    for params in search_params:
        url = target_url.format(
            params["keywords"].replace(" ", "%20"),
            params["location"].replace(" ", "%20"),
            params["f_TPR"],
            # params["start"]
        )
        if params["parttime"]:
            url += "&f_JT=P"
        if params["remote"]:
            url += "&f_WT=2"
        print(url)
        url_list.append(url + "&start={}")
    return url_list
    
linkedin_link_constructor(LINKEDIN_JOB_SEARCH_PARAMS)

https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=senior%20data%20scientist&location=San%20Antonio&f_TPR=r604800
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=senior%20data%20scientist&location=united%20states&f_TPR=r604800&f_WT=2
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=senior%20data%20scientist&location=united%20states&f_TPR=r604800&f_JT=P&f_WT=2


['https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=senior%20data%20scientist&location=San%20Antonio&f_TPR=r604800&start={}',
 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=senior%20data%20scientist&location=united%20states&f_TPR=r604800&f_WT=2&start={}',
 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=senior%20data%20scientist&location=united%20states&f_TPR=r604800&f_JT=P&f_WT=2&start={}']

In [5]:
import datetime
datetime.datetime.now().date().isoformat()

'2025-05-26'